## Airport Filtering

In [1]:
import connectionMongoDB as mongo
import json
import pandas as pd
import getLocation as gl
from haversine import haversine, Unit
import geoqueryfunction as gq
import googleNearBy_API as near
from IPython.display import clear_output
import getAirportCoords as air
import folium
import numpy as np

- All information from airports.csv from: https://ourairports.com/data/

In [2]:
db_airp, coll_airp = mongo.connectCollection('companies','airport')

In [3]:
airport_locations = list(coll_airp.find())


- Adding geolocations to the large airports:

In [4]:
for a in airport_locations:
    long=a['longitude_deg']
    lat=a['latitude_deg']
    value = {"$set": {'location':gl.getLocation(long,lat)}}
    coll_airp.update_one(a,value)

In [5]:
len(airport_locations)


615

- Exporting json with schools, starbucks and companies with more than 1M$ raised.

In [6]:
with open('./output/StarbucksSchool_1M.json', 'r') as myfile:
    data=myfile.read()
obj = json.loads(data)
merge3=pd.DataFrame(obj)

In [7]:
merge3_df=pd.DataFrame(merge3)

In [8]:
airports_df=pd.DataFrame(airport_locations)

- Searching with geoquery the nearest airport to each office.

In [9]:
merge3_df['airports']=merge3_df.apply(lambda a: gq.geoquery(coll_airp,a['Longitude'],a['Latitude'],20000),axis=1)


In [10]:
merge3_df.shape

(727, 21)

- Removing from dataframe the officess without airports near.

In [11]:
merge3_df1=merge3_df[merge3_df['airports'].apply(len)!=0]

In [12]:
merge3_df1.head()

,ref,latitude_schools,longitude_schools,geoloc_school,Latitude_starbucks,Longitude_starbucks,geoloc_stb,Name,Category,Description,...,Longitude,Founded Year,Total Money Raised,Raised Amount,Currency,Address,Zip code,City,GeoLocations,airports
0,5dd59811fc1c02fda798c75c,37.798463,-122.403093,"{'type': 'Point', 'coordinates': [-122.4030926...",37.790070,-122.402083,"{'type': 'Point', 'coordinates': [-122.4020826...",Scribd,news,Read Unlimited Books,...,-122.404052,2007.0,$25.8M,25762000.0,USD,539 Bryant Street,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.404052,...","[{'_id': 5dd6b8fcc0882b53f909fd72, 'id': 3744,..."
1,5dd59811fc1c02fda798c75d,37.760581,-122.395844,"{'type': 'Point', 'coordinates': [-122.3958437...",37.767287,-122.393926,"{'type': 'Point', 'coordinates': [-122.3939261...",Digg,news,user driven social content website,...,-122.394523,2004.0,$45M,45000000.0,USD,135 Mississippi St,94107,San Francisco,"{'type': 'Point', 'coordinates': [-122.394523,...","[{'_id': 5dd6b8fcc0882b53f909fd72, 'id': 3744,..."
4,5dd59811fc1c02fda798c760,40.765788,-73.989530,"{'type': 'Point', 'coordinates': [-73.9895302,...",40.759393,-73.985288,"{'type': 'Point', 'coordinates': [-73.985288, ...",MeetMoi,social,Mobile Dating,...,-73.985506,2007.0,$5.58M,5575000.0,USD,,,New York City,"{'type': 'Point', 'coordinates': [-73.985506, ...","[{'_id': 5dd6b8fcc0882b53f909fb6c, 'id': 3643,..."
5,5dd59811fc1c02fda798c761,37.766440,-122.418942,"{'type': 'Point', 'coordinates': [-122.4189417...",37.777090,-122.417298,"{'type': 'Point', 'coordinates': [-122.4172976...",StumbleUpon,web,Content discovery service,...,-122.419204,2002.0,$18.5M,18500000.0,USD,,,San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,...","[{'_id': 5dd6b8fcc0882b53f90a03b5, 'id': 3878,..."
6,5dd59811fc1c02fda798c762,40.735622,-73.999307,"{'type': 'Point', 'coordinates': [-73.9993065,...",40.741662,-74.005080,"{'type': 'Point', 'coordinates': [-74.00507982...",Joost,games_video,None,...,-74.009447,2006.0,$45M,45000000.0,USD,100 5th Ave Fl 6,10011-6903,New York,"{'type': 'Point', 'coordinates': [-74.0094471,...","[{'_id': 5dd6b8fcc0882b53f909fb6c, 'id': 3643,..."


In [13]:
with open("./output/airport_sch_stb_1M.json", "w+") as file:
    file.write(str(merge3_df1))

## Filtering by vegan 

- Removing companies with founded year < 2009, and keeping the ones with no information about founded year and >2009.

In [16]:
filter_2009=(merge3_df1['Founded Year']>=2009)
filter_found_null= (np.isnan(merge3_df1['Founded Year']))

merge3_df1=merge3_df1[filter_2009 | filter_found_null]
#set(total_df1['Founded Year'])

### NOTA: 37 requests a google places API

- Searching for vegan places 600m next to the office

In [17]:
import time
count=0
vegan_rest=[]
for _,row in merge3_df1.iterrows():
    clear_output()
    count+=1
    print(count)
    v=dict()
    lat=row['Latitude']
    long=row['Longitude']
    dat=near.placesearch(lat,long,600,"vegan restaurant")
    if dat['status']!='ZERO_RESULTS' and dat['status']!='REQUEST_DENIED'and dat['status']!='INVALID_REQUEST':
        lt=dat['results'][0]['geometry']['location']['lat']
        lg=dat['results'][0]['geometry']['location']['lng']
        v['ref']=row['ref']
        v['latitude_veg']=lt
        v['longitude_veg']=lg
        v['geoloc_veg']=gl.getLocation(lg,lt)
        vegan_rest.append(v) 
    time.sleep(1)

37


In [18]:
vegan_df=pd.DataFrame(vegan_rest)

- Merging to have companies with schools, starbucks, airports and vegan places near.

In [19]:
total_df=pd.merge(vegan_df,merge3_df1,how="inner",on='ref')

In [20]:
with open("./output/todo_conVegan.json", "w+") as file:
    file.write(str(total_df))

## Evaluating distances to each place from the office

In [22]:
total_df.columns

Index(['ref', 'latitude_veg', 'longitude_veg', 'geoloc_veg',
       'latitude_schools', 'longitude_schools', 'geoloc_school',
       'Latitude_starbucks', 'Longitude_starbucks', 'geoloc_stb', 'Name',
       'Category', 'Description', 'Latitude', 'Longitude', 'Founded Year',
       'Total Money Raised', 'Raised Amount', 'Currency', 'Address',
       'Zip code', 'City', 'GeoLocations', 'airports'],
      dtype='object')

- With haversine calculating the distance from the office to each place.

In [23]:
total_df['stb']=total_df.apply(lambda a: haversine((a['Latitude'],a['Longitude']),(a['Latitude_starbucks'],a['Longitude_starbucks'])),axis=1)
total_df['school']=total_df.apply(lambda a: haversine((a['Latitude'],a['Longitude']),(a['latitude_schools'],a['longitude_schools'])),axis=1)
total_df['vegan']=total_df.apply(lambda a: haversine((a['Latitude'],a['Longitude']),(a['latitude_veg'],a['longitude_veg'])),axis=1)
total_df['airp']=total_df.apply(lambda a: haversine((a['Latitude'],a['Longitude']),(air.getLatAirport(a),air.getLongAirport(a))),axis=1)


In [24]:
total_df['Total_sum']=total_df.apply(lambda s: s['stb']+s['school']+s['vegan']+s['airp'],axis=1)

In [29]:
total_df.shape

(36, 29)

- Sorting by the minimum sum of distances to all the places from each office.

In [30]:
total_df=total_df.sort_values(by=['Total_sum'])

In [31]:
total_df.head()

,ref,latitude_veg,longitude_veg,geoloc_veg,latitude_schools,longitude_schools,geoloc_school,Latitude_starbucks,Longitude_starbucks,geoloc_stb,...,Address,Zip code,City,GeoLocations,airports,stb,school,vegan,airp,Total_sum
27,5dd59811fc1c02fda798ca24,36.104413,-115.136742,"{'type': 'Point', 'coordinates': [-115.1367421...",36.111716,-115.144953,"{'type': 'Point', 'coordinates': [-115.1449531...",36.102983,-115.136809,"{'type': 'Point', 'coordinates': [-115.1368086...",...,None,89173,Las Vegas,"{'type': 'Point', 'coordinates': [-115.1374, 3...","[{'_id': 5dd6b8fcc0882b53f909fb52, 'id': 3631,...",0.361593,0.914673,0.207267,3.184822,4.668355
26,5dd59811fc1c02fda798ca1d,37.335400,-121.890238,"{'type': 'Point', 'coordinates': [-121.8902383...",37.339597,-121.885251,"{'type': 'Point', 'coordinates': [-121.885251,...",37.337341,-121.888437,"{'type': 'Point', 'coordinates': [-121.8884366...",...,97 S. 2nd Street,95113,San Jose,"{'type': 'Point', 'coordinates': [-121.8884451...","[{'_id': 5dd6b8fcc0882b53f90a03cb, 'id': 3883,...",0.277888,0.599456,0.170240,4.730491,5.778076
17,5dd59811fc1c02fda798c935,42.353888,-71.058466,"{'type': 'Point', 'coordinates': [-71.05846629...",42.366190,-71.053404,"{'type': 'Point', 'coordinates': [-71.0534044,...",42.359142,-71.055548,"{'type': 'Point', 'coordinates': [-71.05554804...",...,None,None,Boston,"{'type': 'Point', 'coordinates': [-71.05682, 4...","[{'_id': 5dd6b8fbc0882b53f909f554, 'id': 3422,...",0.108483,0.859937,0.571364,4.283584,5.823369
1,5dd59811fc1c02fda798c826,42.351445,-71.062707,"{'type': 'Point', 'coordinates': [-71.0627071,...",42.366190,-71.053404,"{'type': 'Point', 'coordinates': [-71.0534044,...",42.357727,-71.059220,"{'type': 'Point', 'coordinates': [-71.05922, 4...",...,8 Winter St.,02108,Boston,"{'type': 'Point', 'coordinates': [-71.060864, ...","[{'_id': 5dd6b8fbc0882b53f909f554, 'id': 3422,...",0.265449,1.320438,0.493852,4.672903,6.752642
28,5dd59811fc1c02fda798ca26,42.339797,-71.064981,"{'type': 'Point', 'coordinates': [-71.0649806,...",42.344259,-71.073494,"{'type': 'Point', 'coordinates': [-71.0734936,...",42.342508,-71.074696,"{'type': 'Point', 'coordinates': [-71.07469644...",...,500 Harrison Ave,02118,Boston,"{'type': 'Point', 'coordinates': [-71.065924, ...","[{'_id': 5dd6b8fbc0882b53f909f554, 'id': 3422,...",0.724163,0.675226,0.246289,5.576836,7.222515


- Using folium to map the best office in terms of the locations of a starbucks, a school, and an airport. 

In [84]:
map_city=folium.Map(total_df.iloc[0]['GeoLocations']['coordinates'][::-1], zoom_start=12)

cols={"GeoLocations":["usd","blue"],"geoloc_veg":["cutlery","green"],"geoloc_school":["book","red"],"geoloc_stb":["coffee","orange"]}

for k,v in cols.items():
    folium.Marker(total_df[k].iloc[0]['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(prefix="fa",icon=v[0],color=v[1]), 
                   ).add_to(map_city)
    
folium.Marker(air.getCoordAirport(total_df.iloc[0])[::-1],
             radius=2,
                icon=folium.Icon(icon="plane",color="black")).add_to(map_city)


In [85]:
map_city

In [86]:
total_df.iloc[0]['GeoLocations']['coordinates']

[-115.1374, 36.1062]